In [145]:

# from utils import plot_network, plot_matrix

import networkx as nx
from scipy.sparse import csc_matrix
import scipy.sparse.csgraph as csgraph
import nxmetis
import scipy
import numpy as np
from bbd_matrix import bbd_matrix, block_vector
from lib_numba import *
from partitionutil import admittance_to_BBD, form_bbd

import time

In [103]:
from scipy.sparse import bsr_matrix, save_npz, csr_matrix

J_csr = scipy.sparse.load_npz('csr.npz')
A = J_csr
G = nx.Graph(A)
#e, p = nxmetis.partition(G, 4, recursive = True, options = nxmetis.MetisOptions(rpart=0,niter=1))


In [142]:
from scipy.sparse import random
def sm(n, density = 0.1):
    smatrix = random(n,n, density = density, format = 'csr')
    return smatrix

sm = sm(10) # A random csr sparse matrix with 10 nodes
G = nx.Graph(sm)
e, p = nxmetis.partition(G, 4, recursive = True, options = nxmetis.MetisOptions(rpart=0,niter=1))

TypeError: 'TypeError' object is not subscriptable

In [143]:
# To get the working number of partitions

def admittance_to_BBD(A, num_parts):
    t0 = time.time()

    G2 = nx.Graph(A)
    G = nx.Graph()
    nodess = [x+1 for x in G2.nodes]
    edgess = [(x+1, y+1) for (x,y) in G2.edges]
    G.add_nodes_from(nodess)
    G.add_edges_from(edgess)

    t1 = time.time()

    edgecuts, partitions  = nxmetis.partition(G,num_parts,recursive=True, options=nxmetis.MetisOptions(rpart=0,niter=1))

    t2 = time.time()

    org_matrix = A

    nodes_part =  {}
    outside_node = {}
    for x in range(len(partitions)):
        nodes_part[x] = partitions[x]
        outside_node[x] = []

    t3 = time.time()

    common_edges = []
    partition_edges = {}
    all_partition_edges = []
    for part in nodes_part.keys():
        partition_edges[part] = []
        for e in G.edges():
            if e[0] in nodes_part[part] and e[1] in nodes_part[part]:
                partition_edges[part].append(e)
                all_partition_edges.append(e)
            else:
                if e[0] not in nodes_part[part] and e[1] not in nodes_part[part]:
                    continue
                elif e[0] not in nodes_part[part]:
                    outside_node[part].append(e[0])
                else:
                    outside_node[part].append(e[1])

    t4 = time.time()

    for e in G.edges():
        if e not in all_partition_edges:
            common_edges.append(e)

    t5 = time.time()

    common_nodes = set([e[0] for e in common_edges] +  [e[1] for e in common_edges])

    t6 = time.time()

    part_nodes_nocommon = {}
    part_nodes_common = {}
    for x in nodes_part.keys():
        part_nodes_nocommon[x] = [y for y in nodes_part[x] if y not in common_nodes]
        part_nodes_common[x] = [y for y in nodes_part[x] if y in common_nodes]

    t7 = time.time()

    all_graphs = {}
    all_graphs_edges = {}
    dmatrix = {}
    for x in part_nodes_nocommon.keys():
        all_graphs[x] = G.subgraph(part_nodes_nocommon[x])
        all_graphs_edges[x] = all_graphs[x].edges()
        dmatrix[x] = csc_matrix( all_graphs[x])

    t8 = time.time()

    index_order = []
    for x in all_graphs.keys():
        perm = nxmetis.node_nested_dissection(all_graphs[x], options=nxmetis.MetisOptions(niter=1))
#         perm = all_graphs[x].nodes()
        perm = nxmetis.node_nested_dissection(all_graphs[x], options=nxmetis.MetisOptions(niter=1))
        index_order += perm

    t9 = time.time()

    Gcorner = nx.Graph()
    Gcorner.add_edges_from(common_edges)
    permg = nxmetis.node_nested_dissection(Gcorner)
#     permg = Gcorner.nodes()

    index_order += permg

    t10 = time.time()

    index_order = np.array(index_order) - 1
    #final_matrix = org_matrix[index_order[:,None],index_order]
    final_matrix = org_matrix.tocsr()[index_order[:, None], index_order]

    t11 = time.time()

    corner_begin = np.sum([len(all_graphs[x].nodes) for x in all_graphs.keys()])
    begin = 0
    blocks = {}
    for x in all_graphs.keys():
        blocks['diagonal_{}'.format(x)] = final_matrix[begin: begin+len(all_graphs[x].nodes()):1, begin: begin+len(all_graphs[x].nodes()):1]
        blocks['lower_{}'.format(x)] = final_matrix[corner_begin::1,begin: begin+len(all_graphs[x].nodes()):1]
        blocks['upper_{}'.format(x)] = final_matrix[begin: begin+len(all_graphs[x].nodes()):1, corner_begin::1]
        begin += len(all_graphs[x].nodes())
    blocks['corner'] = final_matrix[begin: begin+len(Gcorner.nodes()):1, begin: begin+len(Gcorner.nodes()):1]

    t12 = time.time()

    N = np.max(list(nodes_part.keys()))+2

    BBD = bbd_matrix(N)
    for x in blocks.keys():
        if "diagonal" in x: 
            idx = int(x.split("_")[-1])
            BBD.add_diag_block(csc_matrix(blocks[x]),idx)
        elif "upper" in x: 
            idx = int(x.split("_")[-1])
            BBD.add_right_block(csc_matrix(blocks[x]),idx)
        elif "lower" in x: 
            idx = int(x.split("_")[-1])
            BBD.add_lower_block(csc_matrix(blocks[x]),idx)
        elif "corner"  in x: 
            idx = int(N-1)
            BBD.add_diag_block(csc_matrix(blocks[x]),idx)
        else: 
            print("WARNING unknown block!! --> ",x )

    t13 = time.time()

    elapsed = t13 - t0
    time_str = """
    Graph:        {:10.2e} {:8.2%}
    NxMetis:      {:10.2e} {:8.2%}
    OutN:         {:10.2e} {:8.2%}
    PartE:        {:10.2e} {:8.2%}
    ComE:         {:10.2e} {:8.2%}
    ComN:         {:10.2e} {:8.2%}
    PartN:        {:10.2e} {:8.2%}
    AllGr:        {:10.2e} {:8.2%}
    IdxOrd:       {:10.2e} {:8.2%}
    CrnOrd:       {:10.2e} {:8.2%}
    Final:        {:10.2e} {:8.2%}
    BlkSort:      {:10.2e} {:8.2%}
    BBD:          {:10.2e} {:8.2%}
    Total:        {:10.2e}
    """.format(
        t1 - t0, (t1 - t0)/elapsed,
        t2 - t1, (t2 - t1)/elapsed,
        t3 - t2, (t3 - t2)/elapsed,
        t4 - t3, (t4 - t3)/elapsed,
        t5 - t4, (t5 - t4)/elapsed,
        t6 - t5, (t6 - t5)/elapsed,
        t7 - t6, (t7 - t6)/elapsed,
        t8 - t7, (t8 - t7)/elapsed,
        t9 - t8, (t9 - t8)/elapsed,
        t10 - t9, (t10 - t9)/elapsed,
        t11 - t10, (t11 - t10)/elapsed,
        t12 - t11, (t12 - t11)/elapsed,
        t13 - t12, (t13 - t12)/elapsed,
        elapsed,
    )

    print(time_str)

    BBD.print_summary()
        
    pass

def check_partitions(A, num_parts):
    try:
        BBD, index_order = admittance_to_BBD(A, num_parts)
        return True, BBD, index_order
    except:
        return False, None, None

valid_partitions = []

for nparts in range(1, 17):
    is_valid, BBD, index_order = check_partitions(A, nparts)
    if is_valid:
        valid_partitions.append((nparts, BBD, index_order))
        print(nparts)




    Graph:          6.13e-04   13.04%
    NxMetis:        3.50e-04    7.45%
    OutN:           1.67e-06    0.04%
    PartE:          8.34e-05    1.78%
    ComE:           2.98e-05    0.63%
    ComN:           5.96e-06    0.13%
    PartN:          4.53e-06    0.10%
    AllGr:          5.09e-04   10.83%
    IdxOrd:         9.78e-04   20.80%
    CrnOrd:         2.32e-04    4.95%
    Final:          3.25e-04    6.92%
    BlkSort:        7.07e-04   15.04%
    BBD:            8.61e-04   18.31%
    Total:          4.70e-03
    

        Partition Size:       2
        Total NNZ:           74
        Min Block Size:       1
        Min Block NNZ:        1
        Max Block Size:       1
        Max Block NNZ:        1
        Corner Size:         14
        Corner NNZ:          72
        


In [153]:

# BBD, index_order = admittance_to_BBD(A, num_parts=2)
# array = BBD.to_dense()

# import matplotlib.pyplot as plt # Visualizing it
# plt.figure(figsize = (8,6))
# plt.spy(array, markersize = 5)
# plt.title('BBD Matrix')
# plt.xlabel('Col Index')
# plt.ylabel('Row Index')
# plt.show()

BBD, index_order, inv_order = form_bbd(sm, 2)

Forming BBD...


AttributeError: 'csr_matrix' object has no attribute 'Init_net_G0'